# Intro to Reshapr

MOAD Group Software **Discussion**

?? Aug 2022

This notebook can be viewed as a slideshow by using the 
[RISE](https://rise.readthedocs.io/en/stable/index.html])
slide show extension for Jupyter.

*Note: RISE only works with `jupyter notebook`, not with `jupyter lab` :-(*

If you are working in an up to date clone of the 
[UBC-MOAD/PythonNotes repo](https://github.com/UBC-MOAD/PythonNotes),
you can run the slideshow locally.
To do so:
* create an conda environment containing `jupyter` and `rise` with:
```bash
conda env create -f PythonNotes/reshapr-intro/environment.yaml
```
* start `jupyter notebook`
* open `PythonNotes/reshapr-intro/ReshaprIntroSlides.ipynb`
* use `Alt+r` or the `Enter/Exit RISE Slideshow` toobar button to start/stop the slideshow mode
* use `Space` and `Shift+Space` to navigate forward and backward through the slide cells

## Motivation

* Ocean, climate & atmospheric model outputs are netCDF4 files

* Time series of model variables/fields are interesting and useful

* Variable values for time series are stored across many files (daily-ish or monthly-ish)

* Files are large:
  24 hours, 40 depths levels, 398 x 898 x-y grid points of SalishSeaCast is 2.1 Gb on disk

* ~10 Gb in memory due to ~80% deflation when models write files

* Time series extraction = dealing with multiple files that exceed memory size if all loaded at once,
  and that have a computational cost to simply open due to the necessary data decompression

[Xarray](https://xarray.pydata.org/en/latest/) and [Dask](https://docs.dask.org/en/latest/) Address Those Challenges.

* Xarray provides a high level programming interface for labelled multi-dimensional arrays
  that maps especially well to handling NetCDF-4 model results.

* Dask provides a flexible parallel computing framework for operating on extremely large
  datasets without loading them into memory.

`xarray.open_mfdataset()`!!

* Abstracts away the challenge of operating on tens or hundreds of multi-gigabyte files

* Hides the factor of ~5 expansion of the in-memory size of the data compared to the file sizes

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/SNice.svg/1200px-SNice.svg.png" alt=":-)" width=50/>
... for a while ...

Then, confusion and disappointment:

* Slow, at best, or failure due to exhausting the physical and virtual memory 

<img src="https://cdn-0.emojis.wiki/emoji-pics/google/crying-face-google.png" alt=":-(" width=50/>

Why the Sadness?

* Dask's task graph architecture

* Threads, processes, cluster, schedulers ...

* ... And chunking in conjunction with those things